In [5]:
# !pip install boto3

In [15]:
import pandas as pd
import numpy as np
import re
import time

# AWS stuff
import logging
import boto3
from botocore.exceptions import ClientError

In [48]:
# REMOVE CREDS BEFORE UPLOAD!
# Go to Vocareum --> account details in AWS Educate

region = "us-east-1"
aws_access_key_id = "-----redacted--------"
aws_secret_access_key = "-----redacted--------"
aws_session_token = "-----redacted--------"

# get bucket
s3 = boto3.client('s3', 
                  region_name=region,
                  aws_access_key_id=aws_access_key_id, 
                  aws_secret_access_key=aws_secret_access_key, 
                  aws_session_token=aws_session_token)

In [49]:
def check_buckets(s3):
    
    # Retrieve the list of existing buckets
    response = s3.list_buckets()

    # Output the bucket names
    print('Existing buckets:')

    for bucket in response['Buckets']:
        print(f' {bucket["Name"]}')

In [50]:
check_buckets(s3) #none

Existing buckets:


In [58]:
# aws educate only gives access to us-east-1

def create_bucket(bucket_name, s3_client, region=None):
    """
    defaults to us-east-1 region, no need to specify 
    
    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        s3_client.create_bucket(Bucket=bucket_name)
        
    except ClientError as e:
        logging.error(e) #outputs/prints error
        return False
    
    return True

In [59]:
# NOTE: bucket name has specific naming convention: see link
# https://stackoverflow.com/questions/49548451/invalid-bucket-name-when-creating-s3-bucket-with-aws-sdk

# if you get an error, for location stuff, it might just be cuz your bucket name is not unique
# ie. someone took your bucket name already
create_bucket('samanthatangbucket', s3)


True

In [60]:
check_buckets(s3) #none

Existing buckets:
 samanthatangbucket


In [63]:
df = pd.read_csv('./fremont_images.csv')
df

,image_url,address_url,addresses
0,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,100-Black-Mountain-Cir_Fremont_CA_94536
1,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,100-Black-Mountain-Cir_Fremont_CA_94536
2,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,100-Black-Mountain-Cir_Fremont_CA_94536
3,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,100-Black-Mountain-Cir_Fremont_CA_94536
4,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,100-Black-Mountain-Cir_Fremont_CA_94536
...,...,...,...
159,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,169-Rabbit-Ct_Fremont_CA_94539
160,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,169-Rabbit-Ct_Fremont_CA_94539
161,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,169-Rabbit-Ct_Fremont_CA_94539
162,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,169-Rabbit-Ct_Fremont_CA_94539


In [61]:
def upload_file(file_name, bucket, s3_client, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
        
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [69]:
upload_file("fremont_images.csv", "samanthatangbucket", s3)

True

In [78]:
# will have downloaded to path/name

path = 'df_downloaded.csv' 
s3.download_file('samanthatangbucket', 'fremont_images.csv', path)

In [79]:
pd.read_csv("./df_downloaded.csv")

,image_url,address_url,addresses
0,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,100-Black-Mountain-Cir_Fremont_CA_94536
1,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,100-Black-Mountain-Cir_Fremont_CA_94536
2,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,100-Black-Mountain-Cir_Fremont_CA_94536
3,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,100-Black-Mountain-Cir_Fremont_CA_94536
4,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,100-Black-Mountain-Cir_Fremont_CA_94536
...,...,...,...
159,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,169-Rabbit-Ct_Fremont_CA_94539
160,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,169-Rabbit-Ct_Fremont_CA_94539
161,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,169-Rabbit-Ct_Fremont_CA_94539
162,https://ap.rdcpix.com/33cb3ce1fda944eb6660f341...,https://www.realtor.com/realestateandhomes-det...,169-Rabbit-Ct_Fremont_CA_94539


References: 

- https://thecodinginterface.com/blog/aws-s3-python-boto3/
- https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-example-creating-buckets.html
- https://medium.com/bilesanmiahmad/how-to-upload-a-file-to-amazon-s3-in-python-68757a1867c6